In [ ]:
#EXPERIMENT FOR COMPANY A , B , C 
#need to change path to json file for each company everytime that need to run this code
#in this demo, we use the model to run 9 test images folder , 3 folders of images from each company
#dataset of test images available in google drive(test dataset)
#https://drive.google.com/drive/folders/1qtN8mNCxzts1BvIVrpshjnRjpd3C1jLD?usp=sharing

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import cv2
import torch
from PIL import Image
import pandas as pd

### ** Helmet Detection Project Workflow**


0.load the model (.pt) , from training part

1.detect multiple image from a folder

2.load previous saved data from json file ()

3.get the new data in this particular test

4.update the data and send back to to the json file (Company report)

5.upload json file to Pinata IPFS , obtain a CID link

6.write CID link to another json file . SafetyReport

In [ ]:

# load model.pt from previous training
# make sure to set the correct directory to the model.pt , json file,  and the datset 
# the direction of the saved predicted images can be changed in the detect.py file , from the clone folder 

import json
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/helmet_detect/best_yet.pt', force_reload=True)

# set the path to directory of the test folder
directory="/content/drive/My Drive/helmet_detect/demo/companyC/pic3/"

##run prediction , will output the predicted image with boundig box , can change the output directory in detect.py
!python "/content/drive/My Drive/helmet_detect/yolov5/detect.py" --source "/content/drive/My Drive/helmet_detect/demo/companyC/pic3/*.jpg" --weights "/content/drive/MyDrive/helmet_detect/best_yet.pt"


##run prediction, will output string data/information about the prediction

print("  ")
print("  ")
print("***RESULT******RESULT******RESULT******RESULT******RESULT***")
print("  ")
print("  ")
pic=0
import pathlib
for filepath in pathlib.Path(directory).glob('**/*'):
    pic+=1
    print("****************TEST FROM PICUTRE NUMBER{} ****************".format(pic))
    print(filepath.absolute())
    results = model(filepath.absolute())
    #print(results)
    #results.print() 
    #results.xyxy[0] 
    #results.pandas().xyxy[0]
    #print(results.pandas().xyxy[0])
    #results.pandas().xyxy[0].value_counts('name')
    #print(results.pandas().xyxy[0].value_counts('name'))

    totalpeople = len(results.pandas().xyxy[0])
    i=0
    g=0
    b=0
    sp=0
    totalconfi=0
    while i < totalpeople:
      xmin,ymin,xmax,ymax,confi,cla = results.xyxy[0][i].cpu().numpy()
      if cla==0:g+=1
      if cla==1:b+=1
      totalconfi+=confi
      i+=1
    averageconfi=totalconfi/totalpeople
    sp=g/totalpeople

    print("  ")
    print("PREVIOUS RECORD")

    #x = open('/content/drive/My Drive/helmet_detect/data.json')
    x = open('/content/drive/My Drive/helmet_detect/demo/companyC/companyC.json')
    olddata = json.load(x)
    type(olddata)
    SCVisionReport = olddata.get('SCVisionReport')
    type(olddata)
    old_total_people= olddata.get('SCVisionReport').get('totalObserved')
    old_g=olddata.get('SCVisionReport').get('safeObservation')
    old_b=olddata.get('SCVisionReport').get('unsafeObservation')
    old_averageconfi=olddata.get('SCVisionReport').get('DetectionConfidenceScore')
    old_num_test=olddata.get('SCVisionReport').get('totalPics')
    old_sp=olddata.get('SCVisionReport').get('safetyPerformance')




    old_num_test = old_num_test or 0

    if old_num_test>0: old_num_test=old_num_test
    old_total_people = old_total_people or 0
    old_g = old_g or 0
    old_b = old_b or 0
    old_averageconfi = old_averageconfi or 0
    old_sp=old_sp or 0
    if old_total_people==0: old_num_test==0

    print("Previous number of observation {} ".format(int(old_num_test)))
    print("Previous Total number of people : ",int(old_total_people))
    print("Previous number of people wearning safety helmet {}.".format(int(old_g)))
    print("Previous number ofpeople not wearning safety helmet {}.".format(int(old_b)))
    print("Previous safety performance {} ".format(float(old_sp)))
    print("Previous Current confidence score of the detection is {} ".format(float(old_averageconfi)))
   
    print("  ")
    print("RESULT FROM THIS IMAGE")
    print("Total number of people in this picture is ",totalpeople)
    print("There are {} people wearning safety helmet.".format(g))
    print("There are {} people not wearning safety helmet.".format(b))
    print("safety performance {} ".format(float(sp)))
    print("The confidence score of the detection in this picture is {} ".format(averageconfi))
  

    

    new_total_people = int(old_total_people)+totalpeople
    new_g= int(old_g) + g
    new_b= int(old_b) + b
    num_test=old_num_test
    if num_test==0:
      new_averageconfi=averageconfi
    else: new_averageconfi= ((old_averageconfi) + (averageconfi))/2
    
    if num_test==0:
      new_sp=sp
    else: new_sp= (new_g/new_total_people)*100

    num_test= num_test+1


    newreport= {
    "SCVisionReport":{'reportID': 'B-1',
    'reportPeriod': '2022-07-01 : 2022-07-08',
    'totalPics': num_test,
     'totalObserved': new_total_people,
    'safeObservation': new_g,
    'unsafeObservation': new_b,
    'exampleSCVOutput': '',
    'safetyPerformance': new_sp,
    'DetectionConfidenceScore': new_averageconfi}}

    olddata.update(newreport)

    with open('/content/drive/My Drive/helmet_detect/demo/companyC/companyC.json', 'w') as f:
      json.dump(olddata, f)


    print("  ")

    print("************NEW UPDATED RECORD************")
    print("Current total number of observation {} ".format(int(num_test)))
    print("Current Total number of people detected ",new_total_people)
    print("Current total number of people wearning safety helmet {}.".format(new_g))
    print("Current total number of people not wearning safety helmet {}.".format(new_b))
    print("Current safety performance {}.".format(new_sp))
    print("Current confidence score of the detection is {} ".format(new_averageconfi))
    print("*************************************************************************")
    print("  ")
    print("  ")
    print("  ")
    print("  ")
    print("  ")
    print("  ")
    print("  ")

print("  ")
print("  ")
print("DETECTION COMPLETED")
print("  ")
print("  ")


####done"""
    

    
    

    

In [ ]:
## user can input their own , pinata api key and pinata secret api key that link to their pinata account

In [ ]:
from requests import Session, Request
from os import walk, path, sep

## this function will return the link to the uploaded file

def upload_json_ipfs(directory):
    # directory is the abs path of dir
    
    #directory = '/Users/reachsak/Desktop/go_jupiter/data.json'
    
    files = [('file',('data.json',open(directory,'rb'),'application/octet-stream'))]
    
    ipfs_url = "https://api.pinata.cloud/pinning/pinFileToIPFS"
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36',
        'pinata_api_key': "cf32561bbc2438043cb0",
        'pinata_secret_api_key': "cc3f7757705866ebbd0492b35b1cbccaee951f0ca75a3d9ad5ea571c2f45a244"
    }
    # directory.split(sep)[-1] is the name of directory

    request = Request(
        'POST',
        ipfs_url,
        headers=headers,
        files=files
    ).prepare()
    response = Session().send(request)
    print(response.request.url)
    print(response.request.headers)
    print(response.request.body)
    print(response.json())
    print("hash of this file is ")
    

    print(response.json()['IpfsHash'])
    
    hash=response.json()['IpfsHash']
    
    #gateway="https://ipfs.io/ipfs/"
    gateway="https://gateway.pinata.cloud/ipfs/"
    website=gateway+response.json()['IpfsHash']
    print("link to the your file on ipfs is ")
    print(website)
    return website

    

In [ ]:
# get the ipfs link for the uploaded file
link = upload_json_ipfs("/content/drive/My Drive/helmet_detect/demo/companyC/companyC.json")

https://api.pinata.cloud/pinning/pinFileToIPFS
{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36', 'pinata_api_key': 'cf32561bbc2438043cb0', 'pinata_secret_api_key': 'cc3f7757705866ebbd0492b35b1cbccaee951f0ca75a3d9ad5ea571c2f45a244', 'Content-Length': '712', 'Content-Type': 'multipart/form-data; boundary=4ed526d9fa985dc169eedcc9e8883994'}
b'--4ed526d9fa985dc169eedcc9e8883994\r\nContent-Disposition: form-data; name="file"; filename="data.json"\r\nContent-Type: application/octet-stream\r\n\r\n{"status": 1, "metaData": {"name": "Company C", "companyType": "contractor", "city": "Tehran", "projectName": "Tehran Bridge", "workingTimeFrom": "9AM", "workingTimeTo": "5PM"}, "visionInfo": {"number": 3, "registerDate": "2022-06-30"}, "SCVisionReport": {"reportID": "B-1", "reportPeriod": "2022-07-01 : 2022-07-08", "totalPics": 25, "totalObserved": 125, "safeObservation": 121, "unsafeObservation": 4, "exampleSCVOutput

In [ ]:
# create a dictionary for the updated link
#Company A account address: 0xb0a8416D9E6e30CeAD6a598Eef4Cc88d56b1EBAe
#Company B account address: 0xA17Dab0082162079409fD87747CF3eBAd4BfD534
#Company C account address: 0x4c8Db719e7037919647a59E380397A55c53dE4E4

updated_CID_link= {
    "id": 1,
    "status": 0,
    "reportNum": 3,
    "companyAddress": "0x4c8Db719e7037919647a59E380397A55c53dE4E4",
    "reportCID": link
  }



In [ ]:
import json
s = open('/content/drive/My Drive/helmet_detect/demo/safeReport.json')

In [ ]:
# load data of the previous safety report
sdata=json.load(s)

In [ ]:
#append new data
sdata.append(updated_CID_link)
with open('/content/drive/My Drive/helmet_detect/demo/safeReport.json', 'w') as f:
      json.dump(sdata, f)

In [3]:
#check the new updated data on safety report
#the last link is the most up to date link for that data
import json
#below CID link show the sample of the generated report for each detections (9 reports in totals in this demo)
s = open('/Users/reachsak/mainml/computerVisionModule/demo/safeReport.json')
sdata=json.load(s)
sdata

[{'id': 1,
  'status': 0,
  'reportNum': 1,
  'companyAddress': '00xb0a8416D9E6e30CeAD6a598Eef4Cc88d56b1EBAe',
  'reportCID': 'https://gateway.pinata.cloud/ipfs/QmT64ufZzYvFHmhSinP2YNhCgHgNDnL1duzrm2rFk86Yrj'},
 {'id': 1,
  'status': 0,
  'reportNum': 2,
  'companyAddress': '00xb0a8416D9E6e30CeAD6a598Eef4Cc88d56b1EBAe',
  'reportCID': 'https://gateway.pinata.cloud/ipfs/QmQG8nhhTzPYq5K9RSrifgceSRSM4d2hzDx8eDPqSwSMSn'},
 {'id': 1,
  'status': 0,
  'reportNum': 3,
  'companyAddress': '00xb0a8416D9E6e30CeAD6a598Eef4Cc88d56b1EBAe',
  'reportCID': 'https://gateway.pinata.cloud/ipfs/QmUTZFVf58oKNab9HhCCt4tB2GnnEJfyiq8JJGyxUmfAvQ'},
 {'id': 1,
  'status': 0,
  'reportNum': 1,
  'companyAddress': '0xA17Dab0082162079409fD87747CF3eBAd4BfD534',
  'reportCID': 'https://gateway.pinata.cloud/ipfs/QmYNDftrD1hy1U4EkHhfQqLZGvH5nB6cYAGvoZKSntzPMQ'},
 {'id': 1,
  'status': 0,
  'reportNum': 2,
  'companyAddress': '0xA17Dab0082162079409fD87747CF3eBAd4BfD534',
  'reportCID': 'https://gateway.pinata.cloud/ip